In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

from PIL import Image
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Activation , Conv2D , MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.metrics import categorical_crossentropy
from keras.optimizers import Adam
from keras import models
from keras.layers.core import Dense , Flatten , Dropout 
from imgaug import augmenters as iaa

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
import cv2 
print(os.listdir("../input"))

%matplotlib inline
# Any results you write to the current directory are saved as output.

In [ ]:
train_names = "..//input//train.csv"
predict_names = "..//input//sample_submission.csv"
train_imgs = "..//input//train//" 
predict_imgs = "..//input//test//"

train_names = pd.read_csv(train_names)
train_names.head()

In [ ]:
trainX , testX , trainY , testY = train_test_split(train_names['Id'], train_names['Target'], test_size=0.3 , random_state=42)
# print(len(trainX)); print(len(testX))

In [ ]:
# Doing the operation with images
def merging_imgs(img_size , name_of_image, train_predict_imgs_path):
    red_path = train_predict_imgs_path + str(name_of_image+"_red.png")
    green_path = train_predict_imgs_path + str(name_of_image+"_green.png") 
    blue_path = train_predict_imgs_path + str(name_of_image+"_blue.png")
    yellow_path = train_predict_imgs_path + str(name_of_image+"_yellow.png")

    red = cv2.imread(red_path, cv2.IMREAD_UNCHANGED)
    green = cv2.imread(green_path, cv2.IMREAD_UNCHANGED)
    yellow = cv2.imread(yellow_path, cv2.IMREAD_UNCHANGED)
    blue = cv2.imread(blue_path, cv2.IMREAD_UNCHANGED)
    
    img = np.stack((red , green , blue ), axis=-1)
    img = cv2.resize(img , (img_size))
    return img

def data_maker(train_data , target_data, img_size, train_predict_img_dir, no_of_images):
    img_dir = os.listdir(train_predict_img_dir)
    
    train_val_lst = train_data.tolist() 
    target_val_lst = target_data.tolist()
    
    img_lst =[]; target_lst= []
    
#     z =0
    for i in train_data[:int(no_of_images)]:
#         print(z)
        val = train_val_lst.index(i)
        target_val = target_val_lst[val]
        small_lst = target_val.split()
        len_of_target = len(target_val.split())
        
#         z = z +1
        img = merging_imgs(img_size , i, train_predict_img_dir)  
        for i in range(len_of_target):
            if i%2 == 0:
                img_lst.append(img)
                target_lst.append(small_lst[i])
            else:
                img_lst.append(augment(img))
                target_lst.append(small_lst[i])

    return np.array(img_lst) , np.array([int(i) for i in target_lst])


def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)        
        image_aug = augment_img.augment_image(image)
        return image_aug

In [ ]:
test_img_data , test_target_data = data_maker(testX , testY ,(224, 224), train_imgs, 10000) # 224
imgs , targets = data_maker(trainX , trainY , (224, 224), train_imgs, 10000)

# print(test_img[0])
print(len(test_img_data)); print(len(test_target_data))
print(len(imgs)); print(len(targets))

In [ ]:
# Finding the channels in images
# print(imgs)
print(imgs[0].shape)
# print(imgs[0])

# Output layer nurons
output_nurons = len(np.unique(targets))
print(output_nurons)

# Funtion for reshaping the data
def reshaping(data):
    lst = []
    shape = data.shape
    print("shape->", shape)
    for img in data:
        imgs_data = np.resize(img, (shape[1], shape[2], 3))
        lst.append(imgs_data)
        print(imgs_data.shape)
        print(imgs_data[:10])
    return np.array(lst)

In [ ]:
# Now, Displaying the 50 images
for i in range(50):
        if (i % 5) == 0:
            fig, ax = plt.subplots(1,5,figsize=(25,5)) 
        else:
            k = i
            for j in range(5):
                ax[j].imshow(imgs[k])
                k = k + 1

1. ###  Now , Using the One Hot Encoding

In [ ]:
# One Hot Encoding for the target values
def one_hot_encoding(list_of_target):
    one_hot_lst = []
    lst = pd.get_dummies(np.unique(list_of_target))
    for i in list_of_target:
        a = np.array(lst[i]).tolist()
        one_hot_lst.append(a)
    return np.array(one_hot_lst) , len(lst) , lst , np.unique(list_of_target)

print(one_hot_encoding(targets)[0])

### Now , Using the Image Data Genrator

In [ ]:

def image_generator(files,label_file, batch_size): 
    while True:
        index = np.random.choice(len(files),batch_size)

        batch_input = []
        batch_output = [] 
          
        for i in index:
            image = augment(files[i])
            batch_input += [ image ]
            batch_output += [ label_file[i] ]
          # Return a tuple of (input,output) to feed the network
        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        
        yield( batch_x, batch_y )


###  Funtion to load & save Model****

In [ ]:
def saving_model(model_instance,model_name):
    model_instance.save(str(model_name))
    print("Model Saved")
    
def loading_model(model_name):
    model = models.load_model(str(model_name))
    print("Model Loaded")
    return model

###  Making the Model

In [ ]:
from keras.models import Sequential , Model
from keras.layers import BatchNormalization , GlobalAveragePooling2D, GlobalAveragePooling1D, Input
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D , Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
# from keras import backend as K
# K.clear_session()

def model_call(input_shape , output_nurons):
    
    input_tensor = Input(shape= input_shape)
    
#     base_model = keras.applications.vgg16.VGG16(input_tensor = input_tensor, include_top = True , weights = 'imagenet')        
#     base_model = keras.applications.xception.Xception(input_tensor = input_tensor, include_top = True , weights = 'imagenet')
    
    base_model = InceptionV3(input_tensor = input_tensor, include_top = True , weights = 'imagenet')
#     for layer in base_model.layers[:249]:
#         layer.trainable = False
        
    x = base_model.output
    x =Dense(1024 , activation = 'relu')(x)
    predict = Dense(output_nurons , activation = 'sigmoid')(x)
    model = Model(inputs = input_tensor , outputs= predict)
    return model

model = model_call((224,224, 3), 28)
model.summary()
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


train_img = imgs; train_target = one_hot_encoding(targets)[0]
test_img = test_img_data; test_target = one_hot_encoding(test_target_data)[0]

history = model.fit_generator(image_generator(train_img, train_target, 32),
                    steps_per_epoch=int(len(train_img) / 32),
                    validation_data=image_generator(test_img, test_target, 32),
                    validation_steps=int(len(test_img) / 32),
                    epochs=500)

In [ ]:
# Saving the model 
# saving_model(model , "model_v2.model")

In [ ]:
print(history.history.keys())

In [ ]:
# plt
plt.plot(history.epoch, history.history['acc'] , label="acc")
plt.plot(history.epoch, history.history['val_acc'] , label = "val_acc")
plt.legend()
plt.show()

In [ ]:
# plt
plt.plot(history.epoch, history.history['loss'] , label = "loss")
plt.plot(history.epoch, history.history['val_loss'] , label = "val_loss")
plt.legend()
plt.show()

In [ ]:
# Now Making the Prediction and Evaluating the model test_images , atest_target 
score = model.evaluate(test_img ,test_target, verbose = 0)
print("%s: %.2f%%" % ("acc", score[1]*100))

In [ ]:
# test_img,  one_hot_encoding(test_target)[0]
prob = model.predict(test_img[:8])
classes = prob.argmax(axis=-1)
print(prob)
print(classes)

In [ ]:
a , b, c, d = one_hot_encoding(test_target_data) 
# a , b, c, d = one_hot_encoding(b_test)  

lst = [str(i) for i in np.array(c)]

original_lst = []
for i in test_target[:8]:
#     print(i)
    val = lst.index(str(i))
    original_lst.append(val)
    
print(np.array(original_lst))
print(classes)

In [ ]:
sns.heatmap(confusion_matrix(classes, original_lst),annot=True,fmt='.5g')

###  Now , Making the prediction 

In [ ]:
predict_csv = pd.read_csv(predict_names)
predict_csv.head()

In [ ]:
predict_target = np.array([str(i) for i in predict_csv["Predicted"]])
print(predict_target)

In [ ]:
# predict_target = [str(i) for i in predict_csv["Predicted"] ]
predictImg, predictTarget  = data_maker(predict_csv['Id'], predict_target, (224, 224), predict_imgs, len(predict_csv['Id']))
print(predictImg.shape)

In [ ]:
prob = model.predict(predictImg)
classes = prob.argmax(axis=-1)
# print(prob)
print(classes)

###  Output .csv file

In [ ]:
df = pd.DataFrame(data = {"Id":predict_csv['Id'] ,
                          "Predicted": classes })
df.to_csv("//kaggle//working//prediction.csv", sep = ",", index = False)